In [65]:

import asyncio
from dataclasses import replace
from langchain_core.prompts import ChatPromptTemplate
from pydantic.v1 import ValidationError
from orchestrator.models.sequence_planner import SequencePlanner
from orchestrator.models.solution_agent import SolutionAgent
from orchestrator.types.plan import Plan
from orchestrator.types.plan_execute_state import PlanEntry, PlanExecuteState
from langgraph.graph import Graph
from typing import TypedDict, Annotated, List
from langchain_core.messages import HumanMessage
from orchestrator.agents.executor.executor import executor
import operator

n_models_planner = 1
n_models_executor = 2


####DEBUG CODE
#1
class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, HumanMessage):
            return {"__human_message__": True, "content": obj.content}
        if isinstance(obj, BaseModel):
            return obj.model_dump()  # Use model_dump() instead of dict()
        try:
            return json.JSONEncoder.default(self, obj)
        except TypeError:
            return str(obj)

def custom_json_decoder(dct):
    if "__human_message__" in dct:
        return HumanMessage(content=dct["content"])
    return dct
####END DEBUG CODE

planner_prompt = ChatPromptTemplate.from_messages([
    ('system', """For the given objective, come up with a simple step by step plan. 
    This plan should involve individual tasks, that if executed correctly will yield the correct answer. 
    Do not add any superfluous steps. 
    The result of the final step should be the final answer. 
    Make sure that each step has all the information needed - do not skip steps."""),
    ('human', '{objective}'),
])

# class State(TypedDict):
#     messages: Annotated[List[HumanMessage], operator.add]
#     plan: List[str]
#     current_step: int
#     n_models_planner: int
#     n_models_executor: int
#     problem_type: str
#     problem_scope: str
#     best_response_backup: List[dict[str, any]]
#     projected_time_to_finish: int
#     response_outputs: List[dict[str, any]]
#     response_outputs_backup: List[List[dict[str, any]]]
#     num_steps_proposed: int

class State(BaseModel):
    messages: List[HumanMessage] = Field(default_factory=list)
    plan: List[str] = Field(default_factory=list)
    current_step: int = 0
    n_models_planner: int
    n_models_executor: int
    problem_type: str
    problem_scope: str
    best_response_backup: List[dict] = Field(default_factory=list)
    projected_time_to_finish: int = 0
    response_outputs: List[dict] = Field(default_factory=list)
    response_outputs_backup: List[List[dict]] = Field(default_factory=list)
    num_steps_proposed: int = 0
    feature_vectors: List = Field(default_factory=list)

async def planner(state: State) -> State:
    planner = SequencePlanner(
        n_models=1, 
        system_task=state['messages'][-1].content, 
        output_schema=Plan, 
        model='gpt-3.5-turbo'
    )
    response = await planner.agent_runnable.ainvoke({
        'agent_scratchpad': '',
        'n_models': 1,
        'system_task': state['messages'][-1].content
    })
    
    try:
        plan: Plan = Plan.validate(response)
        plan_entry_array = [PlanEntry(step=step, sub_steps=None) for step in plan.message]
        
        return {
            **state,
            "plan": plan_entry_array,
            "num_steps_proposed": len(plan_entry_array)
        }
    except ValidationError as ve:
        raise Exception(ve) from ve

workflow = Graph()
workflow.add_node("planner", planner)
workflow.add_node("executor", executor) #lambda x: executor(x)
workflow.add_edge("planner", "executor")
workflow.add_edge("executor", "planner")
workflow.set_entry_point("planner")
app = workflow.compile()

async def run_workflow(input_message: str, n_models_planner: int = 1, n_models_executor: int = 2):
    initial_state = {
        "messages": [HumanMessage(content=input_message)],
        "plan": [],
        "current_step": 0,
        "n_models_planner": n_models_planner,
        "n_models_executor": n_models_executor,
        "problem_type": "deterministic",
        "problem_scope": "open",
        "best_response_backup": [],
        "projected_time_to_finish": 0,
        "response_outputs": [],
        "response_outputs_backup": [],
        "num_steps_proposed": 0,
        "feature_vectors": []  
    }
    
    print("Initial state:", json.dumps(initial_state, cls=CustomJSONEncoder, indent=2))

    async for output in app.astream(initial_state):
        try:
            print("Raw output:", output)

            if isinstance(output, tuple) and len(output) == 2:
                state, feature_vector = output
                state['feature_vectors'] = state.get('feature_vectors', []) + [feature_vector]
            else:
                state = output
            
            print("Final state for iteration:", json.dumps(state, cls=CustomJSONEncoder, indent=2))
            
            if 'messages' in state and len(state['messages']) > 1:
                print(f"Step {state.get('current_step', 'N/A')}: {state['messages'][-1].content}")
            
            print(f"Current n_models_planner: {state.get('n_models_planner', 'N/A')}, "
                f"n_models_executor: {state.get('n_models_executor', 'N/A')}")
            
            if 'feature_vectors' in state and state['feature_vectors']:
                print(f"Latest feature vector: {state['feature_vectors'][-1]}")
        
        except Exception as e:
            print(f"Error processing output: {e}")
            print(f"Problematic output: {output}")
            import traceback
            print(traceback.format_exc())

async def run_as_main():
    await run_workflow("Order a vegetarian pizza in Munich and have it delivered to Arcisstraße 21, Munich. A human will pay the delivery person upon arrival.", 
                       n_models_planner=1, n_models_executor=2)

# Run the main function
asyncio.run(run_as_main())

Initial state: {
  "messages": [
    {
      "__human_message__": true,
      "content": "Order a vegetarian pizza in Munich and have it delivered to Arcisstra\u00dfe 21, Munich. A human will pay the delivery person upon arrival."
    }
  ],
  "plan": [],
  "current_step": 0,
  "n_models_planner": 1,
  "n_models_executor": 2,
  "problem_type": "deterministic",
  "problem_scope": "open",
  "best_response_backup": [],
  "projected_time_to_finish": 0,
  "response_outputs": [],
  "response_outputs_backup": [],
  "num_steps_proposed": 0,
  "feature_vectors": []
}


/Users/Lukas_1/anaconda3/envs/myenv/lib/python3.12/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


Raw output: {'planner': {'messages': [HumanMessage(content='Order a vegetarian pizza in Munich and have it delivered to Arcisstraße 21, Munich. A human will pay the delivery person upon arrival.')], 'plan': [PlanEntry(step='- Find vegetarian pizza options in Munich', sub_steps=None), PlanEntry(step='- Choose a pizza place that delivers to Arcisstraße 21, Munich', sub_steps=None), PlanEntry(step='- Place an order for a vegetarian pizza', sub_steps=None), PlanEntry(step='- Provide delivery instructions', sub_steps=None), PlanEntry(step='- Coordinate payment upon delivery', sub_steps=None)], 'current_step': 0, 'n_models_planner': 1, 'n_models_executor': 2, 'problem_type': 'deterministic', 'problem_scope': 'open', 'best_response_backup': [], 'projected_time_to_finish': 0, 'response_outputs': [], 'response_outputs_backup': [], 'num_steps_proposed': 5, 'feature_vectors': []}}
Final state for iteration: {
  "planner": {
    "messages": [
      {
        "__human_message__": true,
        "con

/Users/Lukas_1/anaconda3/envs/myenv/lib/python3.12/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


TypeError: Object of type 'FieldInfo' is not JSON serializable

In [66]:
import asyncio
import json
from dataclasses import replace
from langchain_core.prompts import ChatPromptTemplate
from pydantic import ValidationError, BaseModel, Field
from orchestrator.models.sequence_planner import SequencePlanner
from orchestrator.models.solution_agent import SolutionAgent
from orchestrator.types.plan import Plan
from orchestrator.types.plan_execute_state import PlanEntry, PlanExecuteState
from langgraph.graph import Graph
from typing import TypedDict, Annotated, List, Union
from langchain_core.messages import HumanMessage
from orchestrator.agents.executor.executor import executor
import operator

import os
from dotenv import load_dotenv

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

n_models_planner = 1
n_models_executor = 2

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, HumanMessage):
            return {"__human_message__": True, "content": obj.content}
        if isinstance(obj, BaseModel):
            return obj.model_dump()  # Use model_dump() instead of dict()
        try:
            return json.JSONEncoder.default(self, obj)
        except TypeError:
            return str(obj)

def custom_json_decoder(dct):
    if "__human_message__" in dct:
        return HumanMessage(content=dct["content"])
    return dct

planner_prompt = ChatPromptTemplate.from_messages([
    ('system', """For the given objective, come up with a simple step by step plan. 
    This plan should involve individual tasks, that if executed correctly will yield the correct answer. 
    Do not add any superfluous steps. 
    The result of the final step should be the final answer. 
    Make sure that each step has all the information needed - do not skip steps."""),
    ('human', '{objective}'),
])

class State(BaseModel):
    messages: List[HumanMessage] = Field(default_factory=list)
    plan: List[str] = Field(default_factory=list)
    current_step: int = 0
    n_models_planner: int
    n_models_executor: int
    problem_type: str
    problem_scope: str
    best_response_backup: List[dict] = Field(default_factory=list)
    projected_time_to_finish: int = 0
    response_outputs: List[dict] = Field(default_factory=list)
    response_outputs_backup: List[List[dict]] = Field(default_factory=list)
    num_steps_proposed: int = 0
    feature_vectors: List = Field(default_factory=list)

async def planner(state: State) -> State:
    planner = SequencePlanner(
        n_models=1, 
        system_task=state['messages'][-1].content, 
        output_schema=Plan, 
        model='gpt-3.5-turbo'
    )
    print(state)
    response = await planner.agent_runnable.ainvoke({
        'agent_scratchpad': '',
        'n_models': 1,
        'system_task': state['messages'][-1].content
    })
    
    try:
        plan: Plan = Plan.parse_obj(response)
        state['plan'] = list(plan.message)
        state['num_steps_proposed'] = len(plan.message)
        updated_state = state

        return updated_state
    except ValidationError as ve:
        raise Exception(ve) from ve

workflow = Graph()
workflow.add_node("planner", planner)
workflow.add_node("executor", executor)
workflow.add_edge("planner", "executor")
workflow.add_edge("executor", "planner")
workflow.set_entry_point("planner")
app = workflow.compile()

async def run_workflow(input_message: str, n_models_planner: int = 1, n_models_executor: int = 2):
    initial_state = {
        "messages": [HumanMessage(content=input_message)],
        "plan": [],
        "current_step": 0,
        "n_models_planner": n_models_planner,
        "n_models_executor": n_models_executor,
        "problem_type": "deterministic",
        "problem_scope": "open",
        "best_response_backup": [],
        "projected_time_to_finish": 0,
        "response_outputs": [],
        "response_outputs_backup": [],
        "num_steps_proposed": 0,
        "feature_vectors": []  
    }
    
    print("Initial state:", json.dumps(initial_state, cls=CustomJSONEncoder, indent=2))

    async for output in app.astream(initial_state):
        try:
            print("Raw output:", output)

            if isinstance(output, tuple) and len(output) == 2:
                state, feature_vector = output
                state['feature_vectors'] = state.get('feature_vectors', []) + [feature_vector]
            else:
                state = output
            
            print("Final state for iteration:", json.dumps(state, cls=CustomJSONEncoder, indent=2))
            
            if 'messages' in state and len(state['messages']) > 1:
                print(f"Step {state.get('current_step', 'N/A')}: {state['messages'][-1].content}")
            
            print(f"Current n_models_planner: {state.get('n_models_planner', 'N/A')}, "
                f"n_models_executor: {state.get('n_models_executor', 'N/A')}")
            
            if 'feature_vectors' in state and state['feature_vectors']:
                print(f"Latest feature vector: {state['feature_vectors'][-1]}")
        
        except Exception as e:
            print(f"Error processing output: {e}")
            print(f"Problematic output: {output}")
            import traceback
            print(traceback.format_exc())

async def run_as_main():
    await run_workflow("Order a vegetarian pizza in Munich and have it delivered to Arcisstraße 21, Munich. A human will pay the delivery person upon arrival.", 
                       n_models_planner=1, n_models_executor=2)

def main():
    asyncio.run(run_as_main())

# Run the main function
asyncio.run(run_as_main())


Initial state: {
  "messages": [
    {
      "__human_message__": true,
      "content": "Order a vegetarian pizza in Munich and have it delivered to Arcisstra\u00dfe 21, Munich. A human will pay the delivery person upon arrival."
    }
  ],
  "plan": [],
  "current_step": 0,
  "n_models_planner": 1,
  "n_models_executor": 2,
  "problem_type": "deterministic",
  "problem_scope": "open",
  "best_response_backup": [],
  "projected_time_to_finish": 0,
  "response_outputs": [],
  "response_outputs_backup": [],
  "num_steps_proposed": 0,
  "feature_vectors": []
}
{'messages': [HumanMessage(content='Order a vegetarian pizza in Munich and have it delivered to Arcisstraße 21, Munich. A human will pay the delivery person upon arrival.')], 'plan': [], 'current_step': 0, 'n_models_planner': 1, 'n_models_executor': 2, 'problem_type': 'deterministic', 'problem_scope': 'open', 'best_response_backup': [], 'projected_time_to_finish': 0, 'response_outputs': [], 'response_outputs_backup': [], 'num_steps

/Users/Lukas_1/anaconda3/envs/myenv/lib/python3.12/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


Raw output: {'planner': {'messages': [HumanMessage(content='Order a vegetarian pizza in Munich and have it delivered to Arcisstraße 21, Munich. A human will pay the delivery person upon arrival.')], 'plan': ['- Find vegetarian pizza options in Munich', '- Choose a pizza place that delivers to Arcisstraße 21, Munich', '- Place an order for a vegetarian pizza', '- Provide delivery instructions', '- Coordinate payment upon delivery'], 'current_step': 0, 'n_models_planner': 1, 'n_models_executor': 2, 'problem_type': 'deterministic', 'problem_scope': 'open', 'best_response_backup': [], 'projected_time_to_finish': 0, 'response_outputs': [], 'response_outputs_backup': [], 'num_steps_proposed': 5, 'feature_vectors': []}}
Final state for iteration: {
  "planner": {
    "messages": [
      {
        "__human_message__": true,
        "content": "Order a vegetarian pizza in Munich and have it delivered to Arcisstra\u00dfe 21, Munich. A human will pay the delivery person upon arrival."
      }
   

/Users/Lukas_1/anaconda3/envs/myenv/lib/python3.12/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


TypeError: Object of type 'FieldInfo' is not JSON serializable

In [75]:
state = ({'messages': [HumanMessage(content='Order a vegetarian pizza in Munich and have it delivered to Arcisstraße 21, Munich. A human will pay the delivery person upon arrival.'), HumanMessage(content="{'solution': {'designated_answer': 'Search online for vegetarian pizza restaurants in Munich or use food delivery apps to find vegetarian pizza options.', 'additional_information': 'You can also check restaurant review websites or ask locals for recommendations.'}, 'steps': None, 'message': None}")], 'system_task': 'Order a vegetarian pizza in Munich and have it delivered to Arcisstraße 21, Munich. A human will pay the delivery person upon arrival.', 'plan': ['Find vegetarian pizza options in Munich', 'Choose a vegetarian pizza to order', 'Place an order for the chosen pizza', 'Provide delivery address: Arcisstraße 21, Munich', 'Arrange for payment upon delivery'], 'current_step': 1, 'n_models_planner': 1, 'n_models_executor': 2, 'problem_type': 'deterministic', 'problem_scope': 'open', 'best_response_backup': [{'index': 0, 'model_type': 'gpt-3.5-turbo', 'agent_type': 'llm', 'act_max_score': 121.92942268853187, 'message': "{'solution': {'designated_answer': 'Search online for vegetarian pizza restaurants in Munich or use food delivery apps to find vegetarian pizza options.', 'additional_information': 'You can also check restaurant review websites or ask locals for recommendations.'}, 'steps': None, 'message': None}"}, {'message': "{'solution': {'designated_answer': 'Search online for vegetarian pizza restaurants in Munich or use food delivery apps to find vegetarian pizza options.', 'additional_information': 'You can also check restaurant review websites or ask locals for recommendations.'}, 'steps': None, 'message': None}"}], 'projected_time_to_finish': 0, 'response_outputs': [], 'response_outputs_backup': [], 'num_steps_proposed': 5, 'feature_vectors': []}, (121.92942268853187, 405.696, 1, 1, 1, [1, 0, 0, 0], 0, 2, 297, 1.0, 0.0))

In [84]:
if (state[0]['current_step'] >= state[0]['num_steps_proposed']):
    print("hello")

TypeError: tuple indices must be integers or slices, not str

## OLD

In [33]:
# Cell 1: Imports and setup
import asyncio
import nest_asyncio
from dataclasses import replace
from langchain_core.prompts import ChatPromptTemplate
from pydantic.v1 import ValidationError
import os
from orchestrator.models.models import planner_model
from orchestrator.models.sequence_planner import SequencePlanner
from orchestrator.models.solution_agent import SolutionAgent
from orchestrator.types.plan import Plan
from orchestrator.types.plan_execute_state import PlanEntry, PlanExecuteState

nest_asyncio.apply()

# Cell 2: Define planner prompt and chain
planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            """For the given objective, come up with a simple step by step plan. \
            This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
            The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.""",
        ),
        ('human', '{objective}'),
    ]
)

planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.""",
        ),
        ('human', '{objective}'),
    ]
)

model = planner_model.with_structured_output(schema=Plan, include_raw=False)

planner_chain = planner_prompt | model


async def plan_step(state: PlanExecuteState):
    planner = SequencePlanner(
        n_models=1, system_task=state.input, output_schema=Plan, model='gpt-4o-mini'
    )
    response = await planner.agent_runnable.ainvoke(
        {'agent_scratchpad': '', 'n_models': 1, 'system_task': state.input}
    )
    try:
        plan: Plan = Plan.validate(response)
        print(plan)
        solution_agent = SolutionAgent(
            n_models=1,
            model='gpt-4o-mini',
        )
        print(plan.message[0])
        solution = await solution_agent.ainvoke(
            {'agent_scratchpad': '', 'n_models': 1, 'task_step': plan.message[0]}
        )
        print(solution)
        plan_entry_array = [
            PlanEntry(step=step, sub_steps=None) for step in plan.message
        ]
        return replace(state, plan=plan_entry_array)
    except ValidationError as ve:
        raise Exception(ve) from ve


async def run_as_main():
    test_state = PlanExecuteState(
        input='Order a vegetarian pizza in Munich and have it delivered to Arcisstraße 21, Munich. A human will pay the delivery person upon arrival.',
        plan=None,
        past_steps=None,
        response=None,
    )
    test_return = await plan_step(test_state)
    print(test_return)

# Cell 5: Execute the main function
await run_as_main()

/Users/Lukas_1/anaconda3/envs/myenv/lib/python3.12/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


message=['Research local pizza delivery services in Munich that offer vegetarian options.', 'Select a pizza delivery service based on reviews and menu options.', "Choose a vegetarian pizza from the selected service's menu.", 'Provide the delivery address: Arcisstraße 21, Munich.', 'Place the order for the vegetarian pizza.', 'Confirm the order details and estimated delivery time.', 'Notify the human about the order and payment method upon delivery.']


/Users/Lukas_1/anaconda3/envs/myenv/lib/python3.12/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


TypeError: Object of type 'FieldInfo' is not JSON serializable

In [31]:
import json
from pydantic import BaseModel, Field, ValidationError
from typing import List

from langchain_core.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import Field

from orchestrator.types.base_model_config import BaseModelConfig

class Solution(BaseModel):
    result: str

class ChatOpenAI:
    def __init__(self, model, temperature, top_p):
        self.model = model
        self.temperature = temperature
        self.top_p = top_p

    def with_structured_output(self, schema, include_raw=False):
        return schema

class SolutionAgent:
    # Define Solution Agent Context
    prompt_template = """
    You are one agent in a multi-agent model of {n_models} agents.
    It is your job to provide constructive responses your team of agents can work with to solve the task at hand.
    Find the solution to the last system output: {task_step}. Your answer should be geared towards solving the task at hand, provide a "solution:" object containing the designated answer. 
    If you find sub_steps are needed to solve the matter, clearly demarcate the .json object as "steps:" and list each with bulletpoints. 

    Here is some useful information: {agent_scratchpad}
    """

    input_variables = ['agent_scratchpad', 'n_models', 'task_step']

    def __init__(
        self,
        n_models: int,
        model: str,
        temperature: int = 0,
        top_p: float = 0.1,
        agent_scratchpad: str = '',
    ):
        self.llm = ChatOpenAI(model=model, temperature=temperature, top_p=top_p)  # type: ignore
        self.n_models = n_models
        self.agent_scratchpad = agent_scratchpad
        self.prompt = PromptTemplate(
            template=self.prompt_template, input_variables=self.input_variables
        )
        self.model = self.llm.with_structured_output(schema=Solution, include_raw=False)
        self.agent_runnable = self.prompt | self.model


# Custom function to serialize objects
def custom_serializer(obj):
    if isinstance(obj, Field):
        return str(obj)
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

# Create a SolutionAgent instance
solution_agent = SolutionAgent(
    n_models=1,
    model='gpt-3.5-turbo',
)

# Attempt to serialize the SolutionAgent object for debugging
try:
    json_string = json.dumps(solution_agent.__dict__, default=custom_serializer)
    print(json_string)
except TypeError as e:
    print(f"Serialization error: {e}")

Serialization error: isinstance() arg 2 must be a type, a tuple of types, or a union


/Users/Lukas_1/anaconda3/envs/myenv/lib/python3.12/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


In [25]:
solution_agent.__dict__

{'llm': <__main__.ChatOpenAI at 0x10d0f8d40>,
 'n_models': 1,
 'agent_scratchpad': '',
 'prompt': PromptTemplate(input_variables=['agent_scratchpad', 'n_models', 'task_step'], template='\n    You are one agent in a multi-agent model of {n_models} agents.\n    It is your job to provide constructive responses your team of agents can work with to solve the task at hand.\n    Find the solution to the last system output: {task_step}. Your answer should be geared towards solving the task at hand, provide a "solution:" object containing the designated answer. \n    If you find sub_steps are needed to solve the matter, clearly demarcate the .json object as "steps:" and list each with bulletpoints. \n\n    Here is some useful information: {agent_scratchpad}\n    '),
 'model': __main__.Solution,
 'agent_runnable': PromptTemplate(input_variables=['agent_scratchpad', 'n_models', 'task_step'], template='\n    You are one agent in a multi-agent model of {n_models} agents.\n    It is your job to prov

## OUTPUTS

In [ ]:
Initial state: {
  "messages": [
    {
      "__human_message__": true,
      "content": "Order a vegetarian pizza in Munich and have it delivered to Arcisstra\u00dfe 21, Munich. A human will pay the delivery person upon arrival."
    }
  ],
  "plan": [],
  "current_step": 0,
  "n_models_planner": 1,
  "n_models_executor": 2,
  "problem_type": "deterministic",
  "problem_scope": "open",
  "best_response_backup": [],
  "projected_time_to_finish": 0,
  "response_outputs": [],
  "response_outputs_backup": [],
  "num_steps_proposed": 0,
  "feature_vectors": []
}
Model config: "first=RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x304d372c0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x3085b9310>, temperature=0.0, model_kwargs={'top_p': 0.1}, openai_api_key=SecretStr('**********'), openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Plan', 'description': 'Plan to follow in future', 'parameters': {'type': 'object', 'properties': {'message': {'description': 'different steps to follow, should be in sorted order', 'type': 'array', 'items': {'type': 'string'}}}, 'required': ['message'], 'additionalProperties': False}}}], 'parallel_tool_calls': False, 'tool_choice': {'type': 'function', 'function': {'name': 'Plan'}}}) last=PydanticToolsParser(first_tool_only=True, tools=[<class 'orchestrator.types.plan.Plan'>])"
Prompt template: 
    You are one agent in a multi-agent model of {n_models} agents. 
    It is your job to provide constructive responses your team of agents can work with to solve the task at hand.
    Provide a first breakdown of tasks in bulletpoints '-' to plan how you would solve the challenge. Your answer should be geared towards solving the task at hand.  
    The current task is {system_task}. Provide .json output as a list of bulletpoints. Do not add anything extra.

    Here is some useful information: {agent_scratchpad}
    
Output schema: {'title': 'Plan', 'description': 'Plan to follow in future', 'type': 'object', 'properties': {'message': {'title': 'Message', 'description': 'different steps to follow, should be in sorted order', 'type': 'array', 'items': {'type': 'string'}}}, 'required': ['message'], 'additionalProperties': False}
Raw output: {'planner': {'messages': [HumanMessage(content='Order a vegetarian pizza in Munich and have it delivered to Arcisstraße 21, Munich. A human will pay the delivery person upon arrival.')], 'plan': [PlanEntry(step='- Find vegetarian pizza options in Munich', sub_steps=None), PlanEntry(step='- Choose a pizza place that delivers to Arcisstraße 21, Munich', sub_steps=None), PlanEntry(step='- Place an order for a vegetarian pizza', sub_steps=None), PlanEntry(step='- Provide delivery instructions', sub_steps=None), PlanEntry(step='- Coordinate payment upon delivery', sub_steps=None)], 'current_step': 0, 'n_models_planner': 1, 'n_models_executor': 2, 'problem_type': 'deterministic', 'problem_scope': 'open', 'best_response_backup': [], 'projected_time_to_finish': 0, 'response_outputs': [], 'response_outputs_backup': [], 'num_steps_proposed': 5, 'feature_vectors': []}}
Final state for iteration: {
  "planner": {
    "messages": [
      {
        "__human_message__": true,
        "content": "Order a vegetarian pizza in Munich and have it delivered to Arcisstra\u00dfe 21, Munich. A human will pay the delivery person upon arrival."
      }
    ],
    "plan": [
      "PlanEntry(step='- Find vegetarian pizza options in Munich', sub_steps=None)",
      "PlanEntry(step='- Choose a pizza place that delivers to Arcisstra\u00dfe 21, Munich', sub_steps=None)",
      "PlanEntry(step='- Place an order for a vegetarian pizza', sub_steps=None)",
      "PlanEntry(step='- Provide delivery instructions', sub_steps=None)",
      "PlanEntry(step='- Coordinate payment upon delivery', sub_steps=None)"
    ],
    "current_step": 0,
    "n_models_planner": 1,
    "n_models_executor": 2,
    "problem_type": "deterministic",
    "problem_scope": "open",
    "best_response_backup": [],
    "projected_time_to_finish": 0,
    "response_outputs": [],
    "response_outputs_backup": [],
    "num_steps_proposed": 5,
    "feature_vectors": []
  }
}
Current n_models_planner: N/A, n_models_executor: N/A
Debug current_task: "<non-serializable: PlanEntry>"
Current task: PlanEntry(step='- Coordinate payment upon delivery', sub_steps=None)
Debug SolutionAgent inputs: {
  "agent_scratchpad": "",
  "n_models": 1,
  "task_step": "<non-serializable: PlanEntry>"
}
Debug SolutionAgent messages: [
  "<non-serializable: SystemMessage>",
  "<non-serializable: HumanMessage>"
]
Debug SolutionAgent raw response: "<non-serializable: LLMResult>"
Debug SolutionAgent parsed_response: "<Pydantic Model: Solution>"
Debug solution_0: {
  "solution": {
    "steps": [
      "Confirm the delivery details with the supplier, including the delivery date and time.",
      "Ensure that the payment method is agreed upon (cash, credit card, etc.).",
      "Prepare the necessary payment documentation (invoice, receipt, etc.) to present upon delivery.",
      "Communicate with the delivery personnel to confirm payment procedures.",
      "Verify the condition of the delivered goods before making the payment.",
      "Process the payment as per the agreed method once the delivery is confirmed satisfactory.",
      "Obtain a confirmation of payment receipt for your records."
    ]
  },
  "steps": null,
  "message": null
}
Debug output_0: {
  "model_type": "gpt-4o-mini",
  "agent_type": "executor",
  "message": "{'solution': {'steps': ['Confirm the delivery details with the supplier, including the delivery date and time.', 'Ensure that the payment method is agreed upon (cash, credit card, etc.).', 'Prepare the necessary payment documentation (invoice, receipt, etc.) to present upon delivery.', 'Communicate with the delivery personnel to confirm payment procedures.', 'Verify the condition of the delivered goods before making the payment.', 'Process the payment as per the agreed method once the delivery is confirmed satisfactory.', 'Obtain a confirmation of payment receipt for your records.']}, 'steps': None, 'message': None}"
}
Debug SolutionAgent inputs: {
  "agent_scratchpad": "",
  "n_models": 1,
  "task_step": "<non-serializable: PlanEntry>"
}
Debug SolutionAgent messages: [
  "<non-serializable: SystemMessage>",
  "<non-serializable: HumanMessage>"
]
Debug SolutionAgent raw response: "<non-serializable: LLMResult>"
Debug SolutionAgent parsed_response: "<Pydantic Model: Solution>"
Debug solution_1: {
  "solution": {
    "steps": [
      "Confirm the delivery details with the supplier, including the delivery date and time.",
      "Ensure that the payment method is agreed upon (e.g., cash, credit card, bank transfer).",
      "Prepare the necessary payment documentation (e.g., invoice, receipt).",
      "Communicate with the delivery personnel to confirm payment procedures upon arrival.",
      "Verify the condition of the delivered goods before making the payment.",
      "Process the payment as per the agreed method once the delivery is confirmed.",
      "Obtain a receipt or confirmation of payment for record-keeping."
    ]
  },
  "steps": null,
  "message": null
}
Debug output_1: {
  "model_type": "gpt-4o-mini",
  "agent_type": "executor",
  "message": "{'solution': {'steps': ['Confirm the delivery details with the supplier, including the delivery date and time.', 'Ensure that the payment method is agreed upon (e.g., cash, credit card, bank transfer).', 'Prepare the necessary payment documentation (e.g., invoice, receipt).', 'Communicate with the delivery personnel to confirm payment procedures upon arrival.', 'Verify the condition of the delivered goods before making the payment.', 'Process the payment as per the agreed method once the delivery is confirmed.', 'Obtain a receipt or confirmation of payment for record-keeping.']}, 'steps': None, 'message': None}"
}
Debug response_outputs: [
  {
    "model_type": "gpt-4o-mini",
    "agent_type": "executor",
    "message": "{'solution': {'steps': ['Confirm the delivery details with the supplier, including the delivery date and time.', 'Ensure that the payment method is agreed upon (cash, credit card, etc.).', 'Prepare the necessary payment documentation (invoice, receipt, etc.) to present upon delivery.', 'Communicate with the delivery personnel to confirm payment procedures.', 'Verify the condition of the delivered goods before making the payment.', 'Process the payment as per the agreed method once the delivery is confirmed satisfactory.', 'Obtain a confirmation of payment receipt for your records.']}, 'steps': None, 'message': None}"
  },
  {
    "model_type": "gpt-4o-mini",
    "agent_type": "executor",
    "message": "{'solution': {'steps': ['Confirm the delivery details with the supplier, including the delivery date and time.', 'Ensure that the payment method is agreed upon (e.g., cash, credit card, bank transfer).', 'Prepare the necessary payment documentation (e.g., invoice, receipt).', 'Communicate with the delivery personnel to confirm payment procedures upon arrival.', 'Verify the condition of the delivered goods before making the payment.', 'Process the payment as per the agreed method once the delivery is confirmed.', 'Obtain a receipt or confirmation of payment for record-keeping.']}, 'steps': None, 'message': None}"
  }
]